In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_teddynote import logging

logging.langsmith("prompt_chaining")

LangSmith 추적을 시작합니다.
[프로젝트명]
prompt_chaining


### 모듈 import

In [2]:
import pandas as pd
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda 
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document
from datetime import datetime
from langchain_community.document_loaders import JSONLoader
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_anthropic import ChatAnthropic
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.schema import BaseOutputParser
import json

# prompt chain test

In [ ]:
def parse_order_output(output: str) -> dict:
    try:
        return json.loads(output)
    except json.JSONDecodeError:
        return {
            "주문_내용": [],
            "수량": [],
            "특별_요청": ["주문 정보를 파싱할 수 없습니다."]
        }

def create_chains(model):
    understand_order = ChatPromptTemplate.from_messages([
        ("system", """
        고객의 주문을 이해하고 필요한 정보를 추출하세요. 주문 내용, 수량, 특별 요청 사항 등을 파악하세요.
        
        이전 대화 내역:
        {chat_history}
        
        출력 형식:
        {{
            "주문_내용": [주문 항목들],
            "수량": [각 항목의 수량],
            "특별_요청": [특별 요청 사항]
        }}
        
        반드시 위의 출력 형식에 맞춰 JSON 형태로 응답해주세요. 앞뒤에 ```json과 같은 마크다운 표시를 하지 마세요.
        """),
        ("human", "{input}")
    ]) | model | StrOutputParser()


    check_menu = ChatPromptTemplate.from_messages([
        ("system", """
        주문 내용을 확인하고 메뉴에 있는지 검증하세요. 가격과 세트 여부도 확인하세요.
        
        출력 형식:
        {{
            "확인된_메뉴": [확인된 메뉴 항목들],
            "가격": [각 항목의 가격],
            "세트_여부": [각 항목의 세트 여부]
        }}
        """),
        ("human", "{input}")
    ]) | model | StrOutputParser()

    suggest_additions = ChatPromptTemplate.from_messages([
        ("system", """
        현재 주문에 추가할 만한 메뉴를 추천하세요. 세트 메뉴 업그레이드나 사이드 메뉴 추가 등을 제안하세요.
        
        출력 형식:
        {{
            "추천_메뉴": [추천 메뉴 항목들],
            "추천_이유": [각 추천 항목의 이유]
        }}
        """),
        ("human", "{input}")
    ]) | model | StrOutputParser()

    summarize_order = ChatPromptTemplate.from_messages([
        ("system", """
        전체 주문 내용을 요약하고 최종 가격을 계산하세요. 주문 완료 여부도 결정하세요.
        
        출력 형식:
        {{
            "주문_요약": [주문 항목 요],
            "총_가격": 총 가격,
            "주문_완료": true/false
        }}
        """),
        ("human", "{input}")
    ]) | model | StrOutputParser()

    return {
        "understand_order": understand_order,
        "check_menu": check_menu,
        "suggest_additions": suggest_additions,
        "summarize_order": summarize_order
    }

def invoke(question, model, memory, retriever):
    context = retriever.invoke(question)
    context_str = "\n".join([doc.page_content for doc in context])
    chat_history = memory.load_memory_variables({})["chat_history"]

    chains = create_chains(model)

    # 단계별 실행
    understood_order = chains["understand_order"].invoke({
        "input": question,
        "chat_history": chat_history
    })

    checked_menu = chains["check_menu"].invoke({
        "input": understood_order,
        "context": context_str
    })
    
    suggestions = chains["suggest_additions"].invoke({
        "input": checked_menu,
        "context": context_str
    })
    
    # 사용자에게 추천 메뉴 확인
    print(f"AI: {suggestions}와 같은 메뉴를 추가로 추천드립니다. 추가하시겠어요? (예/아니오)")
    user_addition = input("고객: ").strip().lower()
    
    if user_addition == '예':
        print("AI: 어떤 메뉴를 추가하시겠어요?")
        additional_order = input("고객: ").strip()
        checked_menu += f"\n추가 주문: {additional_order}"

    order_summary = chains["summarize_order"].invoke({
        "input": f"{checked_menu}\n{suggestions}",
        "context": context_str
    })

    # 최종 주문 확인
    print(f"AI: 최종 주문 내역입니다: {order_summary}")
    print("이대로 주문하시겠어요? (예/아니오)")
    final_confirmation = input("고객: ").strip().lower()

    if final_confirmation != '예':
        return {"전송": True, "응답": "주문을 처음부터 다시 시작하겠습니다."}

    # 최종 응답 생성
    final_response = ChatPromptTemplate.from_messages([
        ("system", """
        주문 내용을 바탕으로 고객에게 친절하게 응답하세요. 주문 확인, 추천 사항, 최종 가격을 포함하세요.
        
        **주문 결과 예시:**
        {{
            "전송": true,
            "응답": "{{llm_response}}"
        }}
        """),
        ("human", "{order_summary}")
    ]) | model | StrOutputParser()

    return final_response.invoke({"order_summary": order_summary})


## 하나씩 테스트

In [188]:
def create_retriever(file_dir):
    docs = [
        Document(
            page_content=json.dumps(obj['page_content'], ensure_ascii=False),
        )
        for obj in json.load(open(file_dir, 'r', encoding='utf-8'))
    ]
    text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=100, chunk_overlap=0)
    split_docs = text_splitter.split_documents(docs)

    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    cache_dir = LocalFileStore(f"./.cache/embeddings/{file_dir.split('/')[-1]}")
    cached_embedder = CacheBackedEmbeddings.from_bytes_store(
        underlying_embeddings=embeddings,
        document_embedding_cache=cache_dir,
        namespace="solar-embedding-1-large",
    )
    vectorstore = FAISS.from_documents(split_docs, cached_embedder)
    faiss = vectorstore.as_retriever(search_kwargs={"k": 4})

    bm25 = BM25Retriever.from_documents(split_docs)
    bm25.k = 2

    ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25, faiss],
        weights=[0.3, 0.7],
        search_type="mmr",
    )
    return ensemble_retriever

retriever = create_retriever("/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/menu_1017.json")

In [141]:
gpt4o_mini = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18", temperature=0.3)
gpt4o = ChatOpenAI(model_name="gpt-4o-2024-08-06") 
claude = ChatAnthropic(model_name="claude-3-5-sonnet-20240620")

In [121]:
memory = ConversationBufferMemory(
            return_messages=True,
            memory_key="chat_history"
        )
def save_context(user_message, ai_message):
    memory.save_context({"input": str(user_message)}, {"output": str(ai_message)})

#### chain #1 Understand order 

In [261]:
understand_order = ChatPromptTemplate.from_messages([
    ("system", """
    이전 대화내역을 참고해 고객의 주문을 이해하고 필요한 정보를 추출하세요. 
    주문 내용, 수량, 특별 요청 사항 등을 파악하세요.
    특별 요청 사항에 포함되는 항목은 주문 내용에서 제외하세요.
    
    
    이전 대화 내역:
    {chat_history}
    
    출력 형식:
    {{
        "주문_내용": [주문 항목들],
        "수량": [각 항목의 수량],
        "특별_요청": [특별 요청 사항]
    }}
    
    반드시 위의 출력 형식에 맞춰 JSON 형태로 응답해주세요. 앞뒤에 ```json과 같은 마크다운 표시를 하지 마세요.
    """),
    ("human", "{input}")
]) | gpt4o_mini | StrOutputParser()

chat_history = memory.load_memory_variables({})["chat_history"]
question =  "매운거 잘 못먹는데 버거중에 추천해줄만한거 있을까?"

understand_order_r = understand_order.invoke({
    "input": question,
    "chat_history": chat_history
})
understand_order_r = json.loads(understand_order_r)

understand_r = {"understand_order": understand_order_r}

In [262]:
print(understand_order_r)

{'주문_내용': [], '수량': [], '특별_요청': ['매운 음식은 피하고 싶음']}


In [276]:
understand_order = ChatPromptTemplate.from_messages([
    ("system", """
    이전 대화내역을 참고해 고객의 주문을 이해하고 필요한 정보를 추출하세요. 
    주문 내용, 수량, 특별 요청 사항 등을 파악하세요.
    특별 요청 사항에 포함되는 항목은 주문 내용에서 제외하세요.
    
    
    이전 대화 내역:
    {chat_history}
    
    출력 형식:
    {{
        "주문_내용": [주문 항목들],
        "수량": [각 항목의 수량],
        "특별_요청": [특별 요청 사항]
    }}
    
    반드시 위의 출력 형식에 맞춰 JSON 형태로 응답해주세요. 앞뒤에 ```json과 같은 마크다운 표시를 하지 마세요.
    """),
    ("human", "{input}")
]) | gpt4o_mini | StrOutputParser()

chat_history = memory.load_memory_variables({})["chat_history"]
question =  "스낵랩도 하나 주고,세트로 불고기버거 미디엄사이즈로 줘 "

understand_order_r = understand_order.invoke({
    "input": question,
    "chat_history": chat_history
})
understand_order_r = json.loads(understand_order_r)

understand_r = {"understand_order": understand_order_r}

In [277]:
print(understand_order_r)

{'주문_내용': ['스낵랩', '불고기버거 세트'], '수량': [1, 1], '특별_요청': ['불고기버거 미디엄 사이즈']}


In [278]:
result = understand_r["understand_order"]
print(result['주문_내용'])
str_result = f"주문 내용: {', '.join(result['주문_내용'])}"
print(str_result)

['스낵랩', '불고기버거 세트']
주문 내용: 스낵랩, 불고기버거 세트


In [279]:
str_result = f"주문 내용: {', '.join(result['주문_내용'])}, 수량: {', '.join(map(str, result['수량']))}, 특별 요청: {', '.join(result['특별_요청'])}"
print(str_result)

주문 내용: 스낵랩, 불고기버거 세트, 수량: 1, 1, 특별 요청: 불고기버거 미디엄 사이즈


#### chain #2 check_menu

In [265]:
check_menu = ChatPromptTemplate.from_messages([
    ("system", """
    주문 내용을 확인하고 메뉴정보에 있는지 검증하세요. 가격과 세트 여부도 확인하세요.
    불확실한 메뉴명이었으나 메뉴정보를 참고한 후 확인된 메뉴 항목은 "확인된_메뉴"에 포함시키고 "확인되지_않은_메뉴"에는 포함시키지 마세요.
    해결되지 못한 특별요청은 확인되지_않은_이유에 기술하세요
    가격, 세트여부가 확인되지 않은 메뉴는 "확인되지_않은_메뉴"에 포함시키고 가격, 세트여부에 관련 정보를 삽입하지 마세요.
    
    <메뉴정보>
    {context}
    </메뉴정보>
    
    출력 형식:
    {{
        "확인된_메뉴": [확인된 메뉴 항목들],
        "가격": [각 항목의 가격],
        "세트_여부": [각 항목의 세트 여부],
        "확인되지_않은_메뉴" : [확인되지 않은 메뉴 항목들],
        "확인되지_않은_이유":[간략하게 확인되지 않는 이유]
    }}
    반드시 위의 출력 형식에 맞춰 JSON 형태로 응답해주세요. 앞뒤에 ```json과 같은 마크다운 표시를 하지 마세요.
    """),
    ("human", "{input}")
]) | gpt4o | StrOutputParser()

result1 = understand_r["understand_order"]
question_str = f"주문 내용: {', '.join(result1['주문_내용'])}, 특별 요청: {', '.join(result1['특별_요청'])}"

context = retriever.invoke(question_str)
context_str = "\n".join([doc.page_content for doc in context])

checked_menu = check_menu.invoke({
    "input": result1,
    "context": context_str
})

checked_menu_r = json.loads(checked_menu)

check_r = {"check_menu" : checked_menu_r}

In [266]:
print(checked_menu_r)

{'확인된_메뉴': [], '가격': [], '세트_여부': [], '확인되지_않은_메뉴': [], '확인되지_않은_이유': ['매운 음식 피하기 요청에 해당하는 메뉴를 식별할 수 없음']}


#### chain #3 suggest_additions

In [ ]:
suggest_additions = ChatPromptTemplate.from_messages([
    ("system", """
    현재 주문에 추가할 만한 메뉴를 추천하세요. 세트 메뉴 업그레이드나 사이드 메뉴 추가 등을 제안하세요.
    
    세트메뉴는 
    
    출력 형식:
    {
        "추천_메뉴": [추천 메뉴 항목들],
        "추천_이유": [각 추천 항목의 이유]
    }
    반드시 위의 출력 형식에 맞춰 JSON 형태로 응답해주세요. 앞뒤에 ```json과 같은 마크다운 표시를 하지 마세요
    """),
    ("human", "{input}")
]) | gpt4o_mini | StrOutputParser()

In [242]:
memory = ConversationBufferMemory(
            return_messages=True,
            memory_key="chat_history"
        )
def save_context(user_message, ai_message):
    memory.save_context({"input": str(user_message)}, {"output": str(ai_message)})

In [243]:
save_context("매콤한거 먹고싶어","스리라차 마요버거와 맥크리스피 스파이시 버거를 추천드릴 수 있어요")
save_context("둘중에 뭐가 더 싸?", "가격이 저렴한 것을 찾으신다면 맥크리스피 스파이시 버거가 6700원으로 더 저렴합니다.")

In [258]:
def guess_menu(question):
    chat_history = memory.load_memory_variables(['chat_history'])
    
    guess_template = ChatPromptTemplate.from_messages([
        ("system",
        """
        Chat History을 참고해 고객의 주문을 이해하고 메뉴명을 파악하세요
        
        출력형식: "빅맥"
        
        반드시 위의 출력 형식에 맞춰 메뉴명만 출력하세요.
        **Chat History:** {chat_history}
        """),
        ("human", "{question}"),
    ]) | gpt4o_mini | StrOutputParser()

    guess_chain = guess_template.invoke({
        "chat_history": chat_history,
        "question": question
    })
    
    return guess_chain

In [259]:
guess_menu("그거로 줘")

'맥크리스피 스파이시 버거'